### Sensitivity Experiments
#### Methodologies for sensitvity assessment
1) LLM based zero-shot/few-shot ("Categorize sensitivity into these 'x' categories")
2) Transformer based PII extraction. Train sensitivity scorer on top of results (map score to category)
3) TF-IDF for topics. Train sensitivity scorer on top of results (map score to category)
#### Explainability
1) LIME/SHAP to identify most influential features

In [ ]:
! pip install pandas
! pip install openai

In [ ]:
# Load CSV files into pandas DataFrames
import pandas as pd
import os

csv_dir = "./csvs"

dfs = {}
for filename in os.listdir(csv_dir):
    path = csv_dir + "/" + filename
    try:
        name_no_ext = filename.split('.')[0]
        dfs[name_no_ext] = pd.read_csv(path)
    except Exception as e:
        print(f"Failed to read {path}: {e}")

In [ ]:
from IPython.display import display

# Print brief summary and show first few rows for each loaded dataframe
for name, df in dfs.items():
    print(f"{name}: {df.shape}")
    display(df.head())

# Prepare Input

In [ ]:
# Prepare LLM input

llm_inputs = {}
for _, label in dfs['resource_label'].iterrows():
  label_type = label['name']
  label_value = label['value']

  input = llm_inputs.setdefault(label['hash_id'], {})
  label_type_values = input.setdefault(label_type + "s", [])
  label_type_values.append(label_value)

In [ ]:
import random

# Remove inputs that don't have much information (likely indexing went wrong)
valid_llm_inputs = {k: v for k, v in llm_inputs.items() if len(v.keys()) >= 2}

NUM_INPUTS=30
keys = list(valid_llm_inputs.keys())
random.seed(42)
random.shuffle(keys)
keys = keys[:NUM_INPUTS]
reduced_inputs = {k: v for k, v in llm_inputs.items() if k in keys}

# LLM Approach

In [ ]:
# LLM query
import json
import openai

# Load config
openai_config_path = "./configs/openai_standard.json"
with open(openai_config_path, "r") as f:
  openai_config = json.loads(f.read())

# LLM client
client = openai.AzureOpenAI(
  azure_endpoint=openai_config['endpoint'],
  api_version=openai_config['api_version'],
  api_key=openai_config['key']
)

In [ ]:
SYSTEM_PROMPT = """You are an expert at assessing document sensitivity levels. You will receive document metadata in JSON format and must categorize each document's sensitivity.

INPUT FORMAT:
{
  "<document_id>": {
    "document_types": ["<document_type_1>", "<document_type_n>"],
    "primary_subjects": ["<primary_subject_1>", "<primary_subject_n>"]
  }
}

Note: Input may include additional fields such as "summary", "names", etc.

SENSITIVITY CATEGORIES:
Categorize each document into one of the following levels:
- Public: Information suitable for public disclosure
- Internal: Information for internal use only
- Confidential: Sensitive business information with limited access
- Restricted: Highly sensitive information (e.g., PII, financial data, trade secrets)

OUTPUT FORMAT:
Return valid JSON that can be parsed by Python's json.loads():
{
  "document_id": "<document_id>",
  "category": "<sensitivity_category>",
  "confidence": "<confidence_percentage>",
  "explanation": "<brief_justification>"
}

EXAMPLE OUTPUT:
{
  "document_id": "142",
  "category": "Restricted",
  "confidence": "80%",
  "explanation": "Contains PII/PCI including SSNs and bank account information"
}

CRITICAL REQUIREMENTS:
- Output must be valid JSON parseable by json.loads()
- Use consistent capitalization for category names (e.g., "Restricted" not "RESTRICTED")
- Include all four fields: document_id, category, confidence, explanation
- Provide clear, concise explanations for your categorization"""

USER_PROMPT = f"""
====START DOCUMENT INFORMATION====
{reduced_inputs}
====END DOCUMENT INFORMATION====
"""

print(USER_PROMPT)
print(SYSTEM_PROMPT)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "developer",
      "content": SYSTEM_PROMPT
    },
    {
      "role": "user",
      "content": USER_PROMPT
    }
  ]
)

In [ ]:
ans = completion.choices[0].message.content
ans = ans[ans.find('['):ans.rfind(']')+1]
json.loads(ans)

In [ ]:
# Convert LLM JSON output to CSV for Google Sheets
# Assumes the LLM JSON contains the four fields: document_id, category, confidence, explanation
# Expands the original input so each member of any array becomes its own column
import json
import pandas as pd
from pathlib import Path
from IPython.display import display
from datetime import datetime

# Ensure we have the LLM answer (ans) available
if 'ans' not in globals():
    if 'completion' in globals():
        ans = completion.choices[0].message.content
        if '[' in ans and ']' in ans:
            ans = ans[ans.find('['):ans.rfind(']')+1]
    else:
        raise NameError("Neither 'ans' nor 'completion' found. Run the LLM completion cell first.")

# Parse JSON directly (we assume valid JSON with the four expected fields)
results = json.loads(ans)

# Normalize results into a list of records
if isinstance(results, list):
    records = results
elif isinstance(results, dict):
    # If dict maps document_id -> {fields}, convert to list
    if all(isinstance(v, dict) for v in results.values()):
        records = []
        for k, v in results.items():
            row = dict(v)
            if 'document_id' not in row:
                row['document_id'] = k
            records.append(row)
    else:
        # Single-record dict with the four fields
        records = [results]
else:
    raise ValueError(f"Unexpected JSON structure: {type(results)}")

# Build DataFrame (expects columns: document_id, category, confidence, explanation)
df = pd.DataFrame(records)

# Attach and expand the original input for each document if available (prefer reduced_inputs)
input_map = globals().get('reduced_inputs') or globals().get('llm_inputs') or {}

# Build expanded input rows
input_rows = []
for _, row in df.iterrows():
    doc_id = row.get('document_id')
    # try keys in multiple forms
    found = None
    for k in (doc_id, str(doc_id)):
        if k in input_map:
            found = input_map[k]
            break
    if found is None:
        # try loose match by string equality
        for k in input_map.keys():
            if str(k) == str(doc_id):
                found = input_map[k]
                break
    if found is None:
        found = {}

    expanded = {'document_id': doc_id}
    # For each key in the input dict, expand lists into numbered columns,
    # keep scalars as-is (strings/numbers)
    for key, val in found.items():
        if isinstance(val, (list, tuple)):
            for i, v in enumerate(val):
                expanded[f"{key}_{i}"] = v
        else:
            expanded[key] = val

    input_rows.append(expanded)

if input_rows:
    inputs_df = pd.DataFrame(input_rows)
    # Ensure both document_id are strings for a reliable merge
    df['document_id'] = df['document_id'].astype(str)
    inputs_df['document_id'] = inputs_df['document_id'].astype(str)
    merged = df.merge(inputs_df, on='document_id', how='left')
else:
    # No inputs to expand; attach an empty placeholder
    merged = df.copy()

# Save CSV ready for Google Sheets upload
out_path = Path("sensitivity_results_" + str(datetime.now()) + ".csv")
merged.to_csv(out_path, index=False)
print(f"Saved CSV to: {out_path.resolve()} (shape: {merged.shape})")

# Display a preview
display(merged.head())

# Expose variables for interactive use
sensitivity_results_df = merged
sensitivity_results_csv = out_path


# Transformer

In [ ]:
! pip install gliner

In [ ]:
def json_to_string(inp):
  str = "| "
  for category, values in inp.items():
    str += category + ": "
    for value in values[:-1]:
      str += value + ", "
    str += values[-1]
    str += " | "
  return str

INDEX = 29
text = f"""
{json_to_string(list(reduced_inputs.values())[INDEX])}
"""

print(list(reduced_inputs.keys())[INDEX])
print(text)

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")

In [ ]:
labels = ["booking number", "personally identifiable information", "driver licence", "person", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number", "financial data"]
entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity)
    print(entity["text"], "=>", entity["label"])

In [ ]:
! pip install transformers==4.50.3

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}
# Get the model predictions
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# Get the predicted labels
predictions = torch.argmax(outputs.logits, dim=-1)
# Convert token predictions to word predictions
encoded_inputs = tokenizer.encode_plus(text, return_offsets_mapping=True, add_special_tokens=True)
offset_mapping = encoded_inputs['offset_mapping']

In [ ]:
def mask_pii(text, aggregate_redaction=True):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert token predictions to word predictions
    encoded_inputs = tokenizer.encode_plus(text, return_offsets_mapping=True, add_special_tokens=True)
    offset_mapping = encoded_inputs['offset_mapping']

    masked_text = list(text)
    is_redacting = False
    redaction_start = 0
    current_pii_type = ''

    for i, (start, end) in enumerate(offset_mapping):
        if start == end:  # Special token
            continue

        label = predictions[0][i].item()
        if label != model.config.label2id['O']:  # Non-O label
            pii_type = model.config.id2label[label]
            if not is_redacting:
                is_redacting = True
                redaction_start = start
                current_pii_type = pii_type
            elif not aggregate_redaction and pii_type != current_pii_type:
                # End current redaction and start a new one
                apply_redaction(masked_text, redaction_start, start, current_pii_type, aggregate_redaction)
                redaction_start = start
                current_pii_type = pii_type
        else:
            if is_redacting:
                apply_redaction(masked_text, redaction_start, end, current_pii_type, aggregate_redaction)
                is_redacting = False

    # Handle case where PII is at the end of the text
    if is_redacting:
        apply_redaction(masked_text, redaction_start, len(masked_text), current_pii_type, aggregate_redaction)

    return ''.join(masked_text)

def apply_redaction(masked_text, start, end, pii_type, aggregate_redaction):
    for j in range(start, end):
        masked_text[j] = ''
    if aggregate_redaction:
        masked_text[start] = '[redacted]'
    else:
        masked_text[start] = f'[{pii_type}]'

print("Aggregated redaction:")
masked_example_aggregated = mask_pii(text, aggregate_redaction=True)
print(masked_example_aggregated)

print("\nDetailed redaction:")
masked_example_detailed = mask_pii(text, aggregate_redaction=False)
print(masked_example_detailed)

In [ ]:
labels = ["names", "personally identifiable information", "financial information", "financial figures", "revenue"]
entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [ ]:
! pip install pandas
! pip install openai

In [ ]:
from IPython.display import display

# Print brief summary and show first few rows for each loaded dataframe
for name, df in dfs.items():
    print(f"{name}: {df.shape}")
    display(df.head())

In [ ]:
SYSTEM_PROMPT = """You are an expert at assessing document sensitivity levels. You will receive document metadata in JSON format and must categorize each document's sensitivity.

INPUT FORMAT:
{
  "<document_id>": {
    "document_types": ["<document_type_1>", "<document_type_n>"],
    "primary_subjects": ["<primary_subject_1>", "<primary_subject_n>"]
  }
}

Note: Input may include additional fields such as "summary", "names", etc.

SENSITIVITY CATEGORIES:
Categorize each document into one of the following levels:
- Public: Information suitable for public disclosure
- Internal: Information for internal use only
- Confidential: Sensitive business information with limited access
- Restricted: Highly sensitive information (e.g., PII, financial data, trade secrets)

OUTPUT FORMAT:
Return valid JSON that can be parsed by Python's json.loads():
{
  "document_id": "<document_id>",
  "category": "<sensitivity_category>",
  "confidence": "<confidence_percentage>",
  "explanation": "<brief_justification>"
}

EXAMPLE OUTPUT:
{
  "document_id": "142",
  "category": "Restricted",
  "confidence": "80%",
  "explanation": "Contains PII/PCI including SSNs and bank account information"
}

CRITICAL REQUIREMENTS:
- Output must be valid JSON parseable by json.loads()
- Use consistent capitalization for category names (e.g., "Restricted" not "RESTRICTED")
- Include all four fields: document_id, category, confidence, explanation
- Provide clear, concise explanations for your categorization"""

USER_PROMPT = f"""
====START DOCUMENT INFORMATION====
{reduced_inputs}
====END DOCUMENT INFORMATION====
"""

print(USER_PROMPT)
print(SYSTEM_PROMPT)

In [ ]:
ans = completion.choices[0].message.content
ans = ans[ans.find('['):ans.rfind(']')+1]
json.loads(ans)

# Contextualizing the data- connecting to audit log

In [ ]:
rar = dfs['resource_auditrecord']
rar[rar['audited_id'] == 2443586]['operation'].unique()
rar[rar['audited_id'] == 2443586][rar['operation'] == "MODIFIED"].sort_values(by="timestamp")

In [354]:
rrn = dfs['resource_resourcenode']
rrn[rrn['id'] == 2444296]
# rrn[rrn['resource_id'] == 813557]

,id,seqnum,timestamp,params,name,integration,parent_id,resource_id,uniqueness
800,2444296,2,2025-09-27 21:01:33.803+00,\x800495b0000000000000008c12696e74656772617469...,1Q2yTl0eXejDDG0RJEgQv8TL8u7JA7nj3,GOOGLE:terasky-production,NaN,814273.0,3


In [ ]:
# Go through resource_label_assignees

rla = dfs["resource_label_assignees"]
rla[rla['hash_id'] == 81617].head()
# rla.head()

In [356]:
rr = dfs['resource_resource']
rr[rr['id'] == 814273.0].sort_values(by="timestamp")

,id,seqnum,timestamp,params,resource_type,priority,hash_id,parent_id,last_discover
297032,814273,6,2025-09-27 21:01:33.803+00,\x800495b0000000000000008c12696e74656772617469...,FILE,NaN,NaN,NaN,2025-09-28 18:23:08.010107+00


In [ ]:
rh = dfs['resource_hash']
rh[rh['id'] == 81531]

In [ ]:
audit_times = rar[rar['audited_id'] == 2443586]['timestamp']

rr = dfs['resource_resource']
rr = rr[rr['id'] == 813557]

# display(rr)
# target_time = "2025-10-08 10:21:52.542+00"
# target_time = datetime.fromisoformat(target_time)
timestamps = list(rr['timestamp'])

for audit_time in audit_times:
  if (audit_time in timestamps):
    display(rar[rar['audited_id'] == 2443586][rar['timestamp'] == audit_time])
    print("FOUND")

In [ ]:
dfs['resource_label'].head()

In [ ]:
df = rr[rr['id'] == 821690]
# len(df[df['resource_type'] == "STREAM"])
df.head()

In [340]:
# Collecting all of the streams topics/access metadata under one parent resource
# 1) Iterate through audit log

'''
"resource_id": {
  metadata...
  history: [
    operation
    time_of_access
    ip
    resource_information
  ]
}
'''
resources = {}

for ar in dfs['resource_auditrecord'].iterrows():
  ar = ar[1]
  timestamp, operation, location, ip, user_id, rn_id = ar['timestamp'], ar['operation'], ar['geolocation'], ar['client_ip'], ar['user_id'], ar['audited_id']
  resource_info = resources.setdefault(rn_id, {})
  resource_history = resource_info.setdefault("history", [])

  # Info from audit record
  event = {
    "timestamp": timestamp,
    "operation": operation,
    "location": location,
    "ip": ip,
    "user_id": user_id,
    "labels": {} # To be filled out next
  }

  # If the event isn't a modification, no need to fetch any labels. Just add event and move on
  if operation != "MODIFIED" and operation != "FILE_UPLOADED":
    resource_history.append(event)
    continue

  # This is a MODIFIED event, lets fetch the labels from DB
  # Get the resource ID from resource node
  rrn = dfs['resource_resourcenode']
  resource_node = rrn[rrn['id'] == rn_id]
  resource_id = resource_node['resource_id']

  # Should only be one...
  if len(resource_id) != 1:
    print("Found more than one resource with ID: ", resource_id)
    continue
  resource_id = resource_id.item()

  # Get the dataframes we need
  rr, rl, rla = dfs['resource_resource'], dfs['resource_label'], dfs['resource_label_assignees']

  # Grab the resource reference by the audit log event
  resource = rr[rr['id'] == resource_id]
  if len(resource) != 1:
    print("Resource length greater than 1: ", resource)
    continue

  # Work our way down to the relevant STREAM, or in other words actual data, relevant for this audit log event
  file_or_stream = resource.squeeze()

  if file_or_stream['resource_type'] != "STREAM":
    # Must be a file
    assert file_or_stream['resource_type'] == "FILE"
    file = file_or_stream
    streams = rr[rr['parent_id'] == file['id']]
    # Get all the streams before event timestamp
    # streams = streams[streams['timestamp'] <= timestamp]
    
    if len(streams) == 0:
      # print("No streams for id: ", file['id'])
      continue
    # Pick the last one!
    if len(streams) == 1:
      stream = streams.iloc[0]
    else:
      filtered_streams = streams[streams['timestamp'] >= timestamp]
      stream = filtered_streams.iloc[0] if len(filtered_streams) > 0 else streams.iloc[0]
  else:
    stream = file_or_stream
  
  hash_id = stream.squeeze()['hash_id']
  labels = rl[rl['hash_id'] == hash_id]
  for label in labels.iterrows():
    label = label[1]
    label_name, label_value = label['name'], label['value']
    label_arr = event['labels'].setdefault(label_name, [])
    label_arr.append(label_value)
  resource_history.append(event)

Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_id, dtype: float64)
Found more than one resource with ID:  Series([], Name: resource_

In [342]:
import pickle
with open("audit_events_for_model.pkl", "wb") as f:
  pickle.dump(resource, f)

# Ask LLM to understand

In [361]:
test_event = sorted(list(resources.values())[0]['history'], key=lambda x: x['timestamp'])